In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb

/Users/falkvandermeirsch/Documents/DSR/env/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
data = pd.read_csv("data/train.csv")

In [3]:
data

,id,is_delayed,year,month,day,sched_dep_time,sched_arr_time,carrier,origin,dest,distance
0,0,0,2013,12,15,1030,1359,UA,EWR,LAX,2454
1,2,1,2013,4,2,1630,1847,DL,LGA,DTW,502
2,4,1,2013,5,13,2055,2215,MQ,EWR,ORD,719
3,7,0,2013,6,1,855,1120,MQ,LGA,ATL,762
4,8,0,2013,9,19,852,1207,UA,EWR,SNA,2434
5,9,0,2013,11,25,630,1018,US,JFK,PHX,2153
6,10,1,2013,9,4,1529,1747,UA,EWR,DEN,1605
7,12,0,2013,5,8,1455,1820,AA,JFK,MIA,1089
8,14,1,2013,4,24,1110,1246,UA,LGA,ORD,733
9,16,1,2013,4,12,900,1019,US,LGA,BOS,184


In [8]:
test = pd.read_csv('data/test.csv')
test

,id,year,month,day,sched_dep_time,sched_arr_time,carrier,origin,dest,distance
0,1,2013,2,6,1130,1430,DL,JFK,MCO,944
1,3,2013,10,30,1450,1745,UA,EWR,MCO,937
2,5,2013,2,10,1935,2145,9E,JFK,CLE,425
3,6,2013,4,20,1626,1857,EV,EWR,ATL,746
4,11,2013,7,10,1845,2115,DL,JFK,PHX,2153
5,13,2013,6,25,1241,1518,B6,EWR,MCO,937
6,15,2013,3,1,829,1103,B6,JFK,JAX,828
7,17,2013,7,20,1935,2256,DL,JFK,SLC,1990
8,18,2013,6,28,1018,1139,UA,EWR,BOS,200
9,19,2013,12,18,1000,1135,MQ,LGA,BNA,764


In [35]:
data = pd.read_csv("data/train.csv")
data["is_test"] = 0
test = pd.read_csv("data/test.csv")
test["is_test"] = 1
data = data.append(test, sort=False)

In [36]:
data.sort_values(by=["year", "month", "day", "sched_dep_time"], inplace=True)

In [15]:
data.origin.unique()

array(['EWR', 'LGA', 'JFK'], dtype=object)

In [37]:
data['G1'] = ""

In [48]:
data_EWR = data[data.origin == 'EWR'].reset_index(drop=True)
data_LGA = data[data.origin == 'LGA'].reset_index(drop=True)
data_JFK = data[data.origin == 'JFK'].reset_index(drop=True)

,id,is_delayed,year,month,day,sched_dep_time,sched_arr_time,carrier,origin,dest,distance,is_test,G1
0,280700,NaN,2013,1,1,515,819,UA,EWR,IAH,1400,1,EWR_G0
1,3538,NaN,2013,1,1,558,728,UA,EWR,ORD,719,1,EWR_G21546
2,77755,0.0,2013,1,1,600,850,B6,EWR,PBI,1023,0,EWR_G33019
3,84227,0.0,2013,1,1,600,937,UA,EWR,SFO,2565,0,EWR_G1
4,87885,1.0,2013,1,1,600,735,MQ,EWR,ORD,719,0,EWR_G28584


In [56]:
for i, row in data_EWR.iterrows():
    data_EWR.at[i,'G1'] = 'EWR_G' + str(i // 3)

In [75]:
data_EWR.head(12)

,id,is_delayed,year,month,day,sched_dep_time,sched_arr_time,carrier,origin,dest,...,G3,G0,G2,G4,G5,G6,G7,G8,G9,G10
0,280700,NaN,2013,1,1,515,819,UA,EWR,IAH,...,0,NaN,0,0,0,0,0,0,0,0
1,3538,NaN,2013,1,1,558,728,UA,EWR,ORD,...,0,inf,0,0,0,0,0,0,0,0
2,77755,0.0,2013,1,1,600,850,B6,EWR,PBI,...,0,inf,1,0,0,0,0,0,0,0
3,84227,0.0,2013,1,1,600,937,UA,EWR,SFO,...,1,inf,1,0,0,0,0,0,0,0
4,87885,1.0,2013,1,1,600,735,MQ,EWR,ORD,...,1,inf,2,1,0,0,0,0,0,0
5,13673,NaN,2013,1,1,600,854,B6,EWR,FLL,...,1,inf,2,1,1,0,0,0,0,0
6,144714,NaN,2013,1,1,600,902,UA,EWR,LAS,...,2,inf,3,1,1,1,0,0,0,0
7,259995,0.0,2013,1,1,607,915,UA,EWR,MIA,...,2,inf,3,1,1,1,1,0,0,0
8,203110,NaN,2013,1,1,608,728,EV,EWR,IAD,...,2,inf,4,2,1,1,1,1,0,0
9,45068,0.0,2013,1,1,610,910,AA,EWR,MIA,...,3,inf,4,2,1,1,1,1,1,0


In [62]:
data_EWR['day_rank'] = data_EWR.groupby(['year', 'month', 'day']).cumcount()

In [67]:
for i in range(2, 11):
    data_EWR['G' + str(i)] = data_EWR['day_rank'] // i

In [76]:
means = data_EWR.groupby(['year', 'month', 'day', 'G3']).mean()['is_delayed']

In [ ]:
data_EWR['G3_delay'] = 

In [88]:
for i in range(2, 11):
    data_EWR['G' + str(i) +'_delay'] = data_EWR['is_delayed'].rolling(window=i, min_periods=1).mean()

In [103]:
roller = data_EWR['is_delayed'].rolling(window=3, min_periods=1).mean()
data_EWR['G' + str(3) +'_delay'] = roller.shift(1)

In [104]:
data_EWR

,id,is_delayed,year,month,day,sched_dep_time,sched_arr_time,carrier,origin,dest,...,G10,G3_delay,G2_delay,G4_delay,G5_delay,G6_delay,G7_delay,G8_delay,G9_delay,G10_delay
0,280700,NaN,2013,1,1,515,819,UA,EWR,IAH,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3538,NaN,2013,1,1,558,728,UA,EWR,ORD,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,77755,0.0,2013,1,1,600,850,B6,EWR,PBI,...,0,NaN,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
3,84227,0.0,2013,1,1,600,937,UA,EWR,SFO,...,0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
4,87885,1.0,2013,1,1,600,735,MQ,EWR,ORD,...,0,0.000000,0.5,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,NaN
5,13673,NaN,2013,1,1,600,854,B6,EWR,FLL,...,0,0.333333,1.0,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,NaN
6,144714,NaN,2013,1,1,600,902,UA,EWR,LAS,...,0,0.500000,NaN,0.500000,0.333333,0.333333,0.333333,0.333333,0.333333,NaN
7,259995,0.0,2013,1,1,607,915,UA,EWR,MIA,...,0,1.000000,0.0,0.500000,0.333333,0.250000,0.250000,0.250000,0.250000,0.000000
8,203110,NaN,2013,1,1,608,728,EV,EWR,IAD,...,0,0.000000,0.0,0.000000,0.500000,0.333333,0.250000,0.250000,0.250000,0.000000
9,45068,0.0,2013,1,1,610,910,AA,EWR,MIA,...,0,0.000000,0.0,0.000000,0.000000,0.333333,0.250000,0.200000,0.200000,0.333333


In [96]:
roller.shift(1)

0              NaN
1              NaN
2              NaN
3         0.000000
4         0.000000
5         0.333333
6         0.333333
7         0.333333
8         0.250000
9         0.250000
10        0.200000
11        0.166667
12        0.285714
13        0.428571
14        0.500000
15        0.400000
16        0.333333
17        0.428571
18        0.500000
19        0.500000
20        0.600000
21        0.600000
22        0.400000
23        0.250000
24        0.400000
25        0.500000
26        0.500000
27        0.400000
28        0.500000
29        0.571429
            ...   
120805    0.500000
120806    0.333333
120807    0.333333
120808    0.333333
120809    0.500000
120810    0.400000
120811    0.333333
120812    0.400000
120813    0.500000
120814    0.250000
120815    0.250000
120816    0.400000
120817    0.400000
120818    0.400000
120819    0.250000
120820    0.333333
120821    0.500000
120822    0.500000
120823    0.500000
120824    1.000000
120825    1.000000
120826    0.